## Bird Species Indentification using BirdNET

### Overview
This notebook uses the BirdNET deep learning model for bird species identification from audio recordings. BirdNET was developed by Cornell University's Lab of Ornithology and can identify over 6,000 bird species worldwide from their vocalizations.

### Birdnet python package
This notebook uses the `birdnet` Python package, which provides programmatic access to the BirdNET model. Some of the functions we can use:

+ `predict_species_within_audio_file()` - Analyze single audio files
+ `predict_species_within_audio_files_mp()` - Batch process multiple files with multiprocessing
+ `location_based_prediction()` - Get species predictions based on geographic location and time

### Requirements

+ The `birdnet` python package is only available up to Python 3.11.
+ To run on CyVerse: Must run in `Jupyter_Lab_PyTorch_CPU` VICE app due to the Python 3.11 requirement. Tested with 8 CPU Cores, 64GB min memory, 128 min disk space.  It might not need this much but was crashing the kernal at some lower amounts


#### Install Birdnet

In [1]:
!pip install -q birdnet --user 

#### Define function for converting multi-channel audio files to mono

In [8]:
import librosa
import soundfile as sf
from pathlib import Path

def batch_convert_to_mono(file_list):
    mono_files = []
    
    for file_path in file_list:
        file_path = Path(file_path)
        
        # Load and convert to mono
        audio, sr = librosa.load(file_path, mono=True)
        
        # Create output filename
        mono_file = file_path.parent / f"{file_path.stem}_mono{file_path.suffix}"
        
        # Save mono version
        sf.write(mono_file, audio, sr)
        mono_files.append(mono_file)
        print(f"Converted {file_path} -> {mono_file}")
    
    return mono_files

#### Convert your audio files to mono

In [9]:
files = ["./data/XC75502-annas-hummingbird.mp3"]
mono_files = batch_convert_to_mono(files)

Converted data/XC75502-annas-hummingbird.mp3 -> data/XC75502-annas-hummingbird_mono.mp3


#### Use BirdNET to make species predictions

In [10]:
import birdnet

# This will fail if run immediately after the conversion to mono (probably still writing to data store)
predictions = birdnet.predict_species_within_audio_files_mp(
  mono_files
)

for result in predictions:
    print(result)

Predicting species: 100%|██████████| 1/1 [00:01<00:00,  1.85s/file]

(PosixPath('data/XC75502-annas-hummingbird_mono.mp3'), OrderedDict([((0.0, 3.0), OrderedDict([('Selasphorus rufus_Rufous Hummingbird', 0.48477015), ("Calypte anna_Anna's Hummingbird", 0.38208944), ('Archilochus alexandri_Black-chinned Hummingbird', 0.27437147), ("Selasphorus sasin_Allen's Hummingbird", 0.17004326)])), ((3.0, 4.513015873015873), OrderedDict([('Passer domesticus_House Sparrow', 0.468447)]))]))
